In [ ]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

train_data_path = "./train/"
val_data_path = "./val/"
test_data_path = "./test/"

# Fix variable name to avoid overwriting transforms module
transform_pipeline = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform_pipeline)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transform_pipeline)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform_pipeline)

# Data loaders
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

# Model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):  # flatten
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialization
model = SimpleNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu'):
    model.to(device)

    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss_value = loss_fn(outputs, targets)  # fixed variable name
            loss_value.backward()
            optimizer.step()
            training_loss += loss_value.data.item()
        training_loss /= len(train_loader)

        model.eval()
        num_correct = 0
        num_examples = 0

        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss_value = loss_fn(outputs, targets)
            valid_loss += loss_value.data.item()
            correct = torch.eq(torch.max(F.softmax(outputs, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]

        valid_loss /= len(val_loader)
        accuracy = num_correct / num_examples * 100

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, Accuracy: {:.2f}%'.format(
            epoch + 1, training_loss, valid_loss, accuracy))

# Run training
train(model, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader)





Epoch: 1, Training Loss: 2.36, Validation Loss: 5.81, Accuracy: 35.19%
Epoch: 2, Training Loss: 2.99, Validation Loss: 1.35, Accuracy: 68.52%
Epoch: 3, Training Loss: 0.87, Validation Loss: 0.88, Accuracy: 51.85%
Epoch: 4, Training Loss: 0.66, Validation Loss: 0.63, Accuracy: 74.07%
Epoch: 5, Training Loss: 0.44, Validation Loss: 0.60, Accuracy: 74.07%
Epoch: 6, Training Loss: 0.39, Validation Loss: 0.62, Accuracy: 75.00%
Epoch: 7, Training Loss: 0.33, Validation Loss: 0.64, Accuracy: 74.07%
Epoch: 8, Training Loss: 0.30, Validation Loss: 0.63, Accuracy: 74.07%
Epoch: 9, Training Loss: 0.26, Validation Loss: 0.64, Accuracy: 74.07%
Epoch: 10, Training Loss: 0.24, Validation Loss: 0.66, Accuracy: 73.15%
Epoch: 11, Training Loss: 0.22, Validation Loss: 0.66, Accuracy: 73.15%
Epoch: 12, Training Loss: 0.19, Validation Loss: 0.67, Accuracy: 73.15%
Epoch: 13, Training Loss: 0.17, Validation Loss: 0.70, Accuracy: 72.22%
Epoch: 14, Training Loss: 0.16, Validation Loss: 0.68, Accuracy: 73.15%
E

In [ ]:
#save model

from PIL import Image
labels = ['cat','fish']
img = Image.open('./val/cat/97967170_ebd8c6bc5d.jpg')
img = transforms(img)
img = img.unsqueeze(0)
prediction = model(img)
prediction = prediction.argmax()
print(labels[prediction])

TypeError: 'module' object is not callable